In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from scipy.stats.stats import pearsonr

In [2]:
df=pd.read_csv("/home/umair/Desktop/Data Science and BI/data mining/occupancy_data/DataTest.csv", )

In [3]:
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,140,02/02/15 14:19,23.7000,26.272,585.200000,749.200000,0.004764,1
1,141,02/02/15 14:19,23.7180,26.290,578.400000,760.400000,0.004773,1
2,142,02/02/15 14:21,23.7300,26.230,572.666667,769.666667,0.004765,1
3,143,02/02/15 14:22,23.7225,26.125,493.750000,774.750000,0.004744,1
4,144,02/02/15 14:23,23.7540,26.200,488.600000,779.000000,0.004767,1


In [4]:
df.isnull().sum()

id               0
date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,id,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
id,1,-0.108773,0.231894,0.0353051,0.0261599,0.0929943,-0.0238578
Temperature,-0.108773,1,0.716988,0.768428,0.870226,0.894548,0.705784
Humidity,0.231894,0.716988,1,0.561905,0.911611,0.951914,0.610764
Light,0.0353051,0.768428,0.561905,1,0.769167,0.693286,0.927949
CO2,0.0261599,0.870226,0.911611,0.769167,1,0.96444,0.76803
HumidityRatio,0.0929943,0.894548,0.951914,0.693286,0.96444,1,0.70033
Occupancy,-0.0238578,0.705784,0.610764,0.927949,0.76803,0.70033,1


In [6]:
df['Occupancy'].value_counts()

0    1693
1     972
Name: Occupancy, dtype: int64

In [7]:
#Change all the non zero features in 1
df.loc[df['Light'] < 370, 'Light'] = 0
df.loc[df['Light'] >= 370, 'Light'] = 1

In [8]:
df['Light'].value_counts()

0.0    1642
1.0    1023
Name: Light, dtype: int64

In [9]:
df['Occupancy'].value_counts()

0    1693
1     972
Name: Occupancy, dtype: int64

In [10]:
#Crossatab between Occupancy and light
pclass_xt = pd.crosstab(df['Light'], df['Occupancy'])
pclass_xt

Occupancy,0,1
Light,,
0.0,1639,3
1.0,54,969


In [11]:
#Create a new dataframe with only the division of date and hour
df1=pd.DataFrame({
    'date': pd.to_datetime(df['date']).dt.date,
    'time': pd.to_datetime(df['date']).dt.hour})

In [12]:
#Adding teh previous dataframe to the main dataframe
df['date']=df1['date']
df['Hour']=df1['time']

In [13]:
#The new dataframe with date and time splitted
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hour
0,140,2015-02-02,23.7000,26.272,1.0,749.200000,0.004764,1,14
1,141,2015-02-02,23.7180,26.290,1.0,760.400000,0.004773,1,14
2,142,2015-02-02,23.7300,26.230,1.0,769.666667,0.004765,1,14
3,143,2015-02-02,23.7225,26.125,1.0,774.750000,0.004744,1,14
4,144,2015-02-02,23.7540,26.200,1.0,779.000000,0.004767,1,14


In [14]:
#Adjusting the order of the column
df=df[['id', 'date', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [15]:
df.head()

,id,date,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,140,2015-02-02,14,23.7000,26.272,1.0,749.200000,0.004764,1
1,141,2015-02-02,14,23.7180,26.290,1.0,760.400000,0.004773,1
2,142,2015-02-02,14,23.7300,26.230,1.0,769.666667,0.004765,1
3,143,2015-02-02,14,23.7225,26.125,1.0,774.750000,0.004744,1
4,144,2015-02-02,14,23.7540,26.200,1.0,779.000000,0.004767,1


In [16]:
#Add the column with the name of the day
df['date'] = pd.to_datetime(df['date'], format='%Y-%d-%m')

df['DayName'] = df['date'].dt.weekday_name

In [17]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [18]:
df.head()

,id,date,DayName,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,140,2015-02-02,Monday,14,23.7000,26.272,1.0,749.200000,0.004764,1
1,141,2015-02-02,Monday,14,23.7180,26.290,1.0,760.400000,0.004773,1
2,142,2015-02-02,Monday,14,23.7300,26.230,1.0,769.666667,0.004765,1
3,143,2015-02-02,Monday,14,23.7225,26.125,1.0,774.750000,0.004744,1
4,144,2015-02-02,Monday,14,23.7540,26.200,1.0,779.000000,0.004767,1


In [19]:
#Add a new column for the boolean variable IsWorkDay, this feature is 1 if the day is a work day, 0 otherwise
df["IsWorkDay"] = np.nan
df['IsWorkDay']=df['IsWorkDay'].astype(bool).astype(int)

In [20]:
#Fill the feature IsWorkDay with the corrsponding 1 or 0
for index, row in df.iterrows():
    if row["DayName"]!="Saturday" and row["DayName"]!="Sunday":
        df.loc[index, "IsWorkDay"] = 1
    else:
        df.loc[index, "IsWorkDay"] = 0

In [21]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'IsWorkDay', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [22]:
#Drop the column id because is useless for us
df.drop(['id'], axis=1, inplace =True)

In [23]:
#Drop the column HumidityRatio because is useless for us
df.drop(['HumidityRatio'], axis=1, inplace =True)

In [24]:
#Final attributes of our cleaned dataset
df.head()

,date,DayName,IsWorkDay,Hour,Temperature,Humidity,Light,CO2,Occupancy
0,2015-02-02,Monday,1,14,23.7000,26.272,1.0,749.200000,1
1,2015-02-02,Monday,1,14,23.7180,26.290,1.0,760.400000,1
2,2015-02-02,Monday,1,14,23.7300,26.230,1.0,769.666667,1
3,2015-02-02,Monday,1,14,23.7225,26.125,1.0,774.750000,1
4,2015-02-02,Monday,1,14,23.7540,26.200,1.0,779.000000,1


In [25]:
#Add a new column for the variable IsWorkHour, this feature is 1 if the hour is a work hour,
#0 if it isn't a work hour and 2 if we have the doubt, so in the boundary values 
df["IsWorkHour"] = np.nan
df['IsWorkHour']=df['IsWorkHour'].astype(bool).astype(int)

In [26]:
#Fill the feature IsWorkHour with the corrsponding 0, 1 or 2
for index, row in df.iterrows():
    if row["Hour"]==13 or row["Hour"]==7 or row["Hour"]==18:
        df.loc[index, "IsWorkHour"] = 2 #could be both a work hour or not
    elif row["Hour"]<7 or row["Hour"]>18:
        df.loc[index, "IsWorkHour"] = 0 #is not a work hour
    else:
        df.loc[index, "IsWorkHour"] = 1 #is a work hour

In [27]:
#Adjusting the order of the column
df=df[['date', 'DayName', 'IsWorkDay', 'Hour', 'IsWorkHour', 'Temperature', 'Humidity', 'Light', 'CO2', 'Occupancy']]

In [28]:
#test for see if everything works
df[1300:3000]

,date,DayName,IsWorkDay,Hour,IsWorkHour,Temperature,Humidity,Light,CO2,Occupancy
1300,2015-02-03,Tuesday,1,11,1,22.200000,27.500000,1.0,1094.000000,1
1301,2015-02-03,Tuesday,1,12,1,22.254000,27.560000,1.0,1092.400000,1
1302,2015-02-03,Tuesday,1,12,1,22.222500,27.575000,1.0,1081.666667,1
1303,2015-02-03,Tuesday,1,12,1,22.245000,27.583333,1.0,1077.000000,1
1304,2015-02-03,Tuesday,1,12,1,22.254000,27.540000,1.0,1074.400000,1
...,...,...,...,...,...,...,...,...,...,...
2660,2015-02-04,Wednesday,1,10,1,24.290000,25.700000,1.0,1150.250000,1
2661,2015-02-04,Wednesday,1,10,1,24.330000,25.736000,1.0,1129.200000,1
2662,2015-02-04,Wednesday,1,10,1,24.330000,25.700000,1.0,1125.800000,1
2663,2015-02-04,Wednesday,1,10,1,24.356667,25.700000,1.0,1123.000000,1


In [29]:
#Export the dataset cleaned
df.to_csv(r'DataCleanedTest.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2665 entries, 0 to 2664
Data columns (total 10 columns):
date           2665 non-null datetime64[ns]
DayName        2665 non-null object
IsWorkDay      2665 non-null int64
Hour           2665 non-null int64
IsWorkHour     2665 non-null int64
Temperature    2665 non-null float64
Humidity       2665 non-null float64
Light          2665 non-null float64
CO2            2665 non-null float64
Occupancy      2665 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 208.3+ KB


In [31]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,IsWorkDay,Hour,IsWorkHour,Temperature,Humidity,Light,CO2,Occupancy
IsWorkDay,nan,nan,nan,nan,nan,nan,nan,nan
Hour,nan,1,0.130411,0.368454,0.347908,0.108153,0.339607,0.102986
IsWorkHour,nan,0.130411,1,0.552159,0.431432,0.603589,0.562105,0.572654
Temperature,nan,0.368454,0.552159,1,0.716988,0.718828,0.870226,0.705784
Humidity,nan,0.347908,0.431432,0.716988,1,0.601242,0.911611,0.610764
Light,nan,0.108153,0.603589,0.718828,0.601242,1,0.770259,0.955152
CO2,nan,0.339607,0.562105,0.870226,0.911611,0.770259,1,0.76803
Occupancy,nan,0.102986,0.572654,0.705784,0.610764,0.955152,0.76803,1
